In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
pd.set_option('display.max_rows', None)
df1 = pd.read_csv("D:/da/result.csv")
df1.head()

In [ ]:
df2 = pd.read_csv("D:/da/betasahm1.csv", header = None)
df2.columns = ["Date", "Post"]
df2.head(10)

In [ ]:
#extract hashtags world
hashtags = df2.Post.str.findall(r'#.*?(?=\s|$)')


#insert the hashtags column in the data 
df2['hashtags'] = hashtags
df2.head()

In [ ]:
#delete empty square bracket
hashtags_list_df = df2.loc[
                       df2.hashtags.apply(
                           lambda hashtags_list: hashtags_list !=[]
                       ),['hashtags']]

In [ ]:
#delete NaN row
hashtags_list_df = hashtags_list_df.dropna()
hashtags_list_df.head(20)

In [ ]:
#insert the hashtags column in the data 
df2["hashtags"] = hashtags_list_df
df2.head()

In [ ]:
#delete NaN rows 
df = df2.dropna()
df.head()

In [ ]:
df = df[['Date', 'hashtags', 'Post']]
df.head()

In [ ]:
pd.set_option("max_colwidth", 2)
df[["Date","hashtags", "Post"]].head(10)

In [ ]:
foo = lambda a: "/".join(a) 

In [ ]:
#merge posts with same hashtags worlds 
df = df.groupby(by=df.hashtags.astype(str)).agg({'Post': foo}).reset_index()
df.head()

In [ ]:
#split rows with more than one hashtag word and set post for them

new_df = pd.DataFrame(df["hashtags"].str.split(" ",15).tolist(), index=df["Post"]).stack()
new_df = new_df.reset_index([0, "Post"])

new_df.columns = ["Post", "hashtags"]
new_df = new_df[["hashtags", "Post"]]

new_df.head()

In [ ]:
#clean the hashtags colum

import re
import string

def clean_text_round1(text):
    text = text.lower()
    #text = re.sub('\[.*?\]', '', str(text))
    text = re.sub('[%s]' % re.escape(string.punctuation), '', str(text))
    text = re.sub('\w*\d\w*', '', str(text))
    return text

round1 = lambda x: clean_text_round1(x) 

In [ ]:
new_df.hashtags = new_df.hashtags.astype(str)

In [ ]:
data_clean = pd.DataFrame(new_df.hashtags.apply(round1))
data_clean.tail(30)

In [ ]:
data_clean["Post"] = new_df.Post

In [ ]:
data_clean.head(30)

In [ ]:
foo = lambda a: "/".join(a) 

In [ ]:
#merge posts with same hashtags worlds 
df = data_clean.groupby(by=data_clean.hashtags.astype(str)).agg({'Post': foo}).reset_index()
df.head()

In [ ]:
#delete fake hashtags word
df = df.drop([0,1,13,18,25])

In [ ]:
df.head()

In [ ]:
# Apply a first round of cleaning
import re
import string

def clean_text_round1(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', str(text))
    text = re.sub('[%s]' % re.escape(string.punctuation), '', str(text))
    text = re.sub('\w*\d\w*', '', str(text))
    return text

round1 = lambda x: clean_text_round1(x) 

In [ ]:
df.Post = df.Post.astype(str)

In [ ]:
data_clean = pd.DataFrame(df.Post.apply(round1))
data_clean.head()

In [ ]:
# Apply a second round of cleaning
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [ ]:
data_clean["hashtags"] = df.hashtags

In [ ]:
df = data_clean[['hashtags', 'Post']]
df.head(30)

In [ ]:
# Let's pickle it for later use
df.to_pickle("corpus.pkl")

In [ ]:
# We are going to create a document-term matrix using CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
data_cv = cv.fit_transform(data_clean.Post)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = data_clean.hashtags
data_dtm.head()

In [ ]:
# Let's pickle it for later use
data_dtm.to_pickle("dtm.pkl")

In [ ]:
# Let's also pickle the cleaned data (before we put it in document-term matrix format) and the CountVectorizer object

import pickle
df.to_pickle('data_clean.pkl')
pickle.dump(cv, open("cv.pkl", "wb"))